In [1]:
import pandas as pd
import numpy as np
import os 
import time

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

def data_generate(data_pm25_filled):
    data = []
    label = []
    data_pm25_filled_array = data_pm25_filled.iloc[:,3:].values
    for i in range(data_pm25_filled_array.shape[0] -12 -6):
        data_now = data_pm25_filled_array[i:i+12,:]
        label_now = data_pm25_filled_array[i+12:i+18,:]
        data.append(data_now)
        label.append(label_now)
    data = np.array(data)
    label = np.array(label)
    return data, label

def Evaluation(label, predict):
    MAE = np.mean(np.abs(label - predict))
    RMSE = np.power(np.mean(np.power(label - predict,2)) ,0.5)

    label_grade = label
    label_grade[label_grade < 35] = 1
    label_grade[label_grade > 250] = 6
    label_grade[label_grade > 150] = 5
    label_grade[label_grade > 115] = 4
    label_grade[label_grade > 75] = 3
    label_grade[label_grade > 35] = 2
    
    predict_grade = predict
    predict_grade[predict_grade < 35] = 1
    predict_grade[predict_grade > 250] = 6
    predict_grade[predict_grade > 150] = 5
    predict_grade[predict_grade > 115] = 4
    predict_grade[predict_grade > 75] = 3
    predict_grade[predict_grade > 35] = 2
    
    res = np.zeros(label_grade.shape)
    res[label_grade == predict_grade] = 1
    num_cor = res.sum()
    num_all = res.shape[0] * res.shape[1]
    prec = num_cor/num_all
    return MAE,RMSE,prec

In [2]:
data_all = pd.read_csv("data_all.csv")
# 选取pm2.5数据
data_all_pm25 = data_all[data_all['type']=='PM2.5']
# 删去无用的前三列
data_all_pm25 = data_all_pm25.iloc[:,3:]
# "就近"填充缺失数据
data_all_pm25_filled = data_all_pm25.fillna(method='ffill')
data_all_pm25_filled = data_all_pm25_filled.fillna(method='bfill')
# 划分数据集
data_train_pm25_filled = data_all_pm25_filled[data_all_pm25_filled['date'] < 20200000]
data_train_pm25_filled = data_train_pm25_filled[data_train_pm25_filled['date'] > 20150000]
data_val_pm25_filled = data_all_pm25_filled[data_all_pm25_filled['date'] > 20200000]
data_test_pm25_filled = data_all_pm25_filled[data_all_pm25_filled['date'] < 20150000]
# 制作时序数据集
train_data,train_label = data_generate(data_train_pm25_filled)
val_data,val_label = data_generate(data_val_pm25_filled)
test_data,test_label = data_generate(data_test_pm25_filled)

/data2/syh/ENTER/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 训练并测试模型
print('training with certain depth')
# models = [DecisionTreeRegressor(),RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor()]
models = []
max_depth = [50,40,30]
for d in max_depth:
    models.append(DecisionTreeRegressor(max_depth=d))

train_predict_all = []
val_predict_all = []
test_predict_all = []

for model_now in models:
    print(model_now)
    start = time.time()
    train_predict = np.zeros(train_label.shape)
    val_predict = np.zeros(val_label.shape)
    test_predict = np.zeros(test_label.shape)
    
    for ti in range(train_label.shape[1]):
        for lo in range(train_label.shape[2]):
            model_now.fit(train_data[:,:,lo], train_label[:,ti,lo])
            train_predict[:,ti,lo] = model_now.predict(train_data[:,:,lo])
            val_predict[:,ti,lo] = model_now.predict(val_data[:,:,lo])
            test_predict[:,ti,lo] = model_now.predict(test_data[:,:,lo])
            
    train_predict_all.append(train_predict)
    val_predict_all.append(val_predict)
    test_predict_all.append(test_predict)
    end = time.time()
    print('training lasted: '+str(end-start))

training with certain depth
DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
training lasted: 124.76740097999573
DecisionTreeRegressor(criterion='mse', max_depth=40, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
training lasted: 124.28532981872559
DecisionTreeRegressor(criterion='mse', max_depth=30, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, m

In [4]:
# 评价模型性能
for m in range(3):
    print(models[m])
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(val_label[:,i,:], val_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))

DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
time:1 MAE = 9.423905543580366 RMSE = 18.52055175053245 PREC = 0.8090220612446494
time:2 MAE = 14.890913069875047 RMSE = 27.91793444212593 PREC = 0.7215108895056211
time:3 MAE = 19.041848183260637 RMSE = 34.18655100757732 PREC = 0.6596641422456372
time:4 MAE = 22.34809235213685 RMSE = 38.49685982304607 PREC = 0.6166988099158004
time:5 MAE = 25.501135680740845 RMSE = 43.45373169150281 PREC = 0.5823227809398372
time:6 MAE = 28.0690352770722 RMSE = 47.09512645513467 PREC = 0.5557081706571334
DecisionTreeRegressor(criterion='mse', max_depth=40, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
         